<a href="https://colab.research.google.com/github/mburkey3/Intro_to_ML/blob/main/Homework5/Homework5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import torch
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
file_path = '/content/drive/My Drive/Intro_to_ML/Datasets/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [17]:
# Binary map Outputs to 0 or 1
# M = 1
# B = 0
# Map function
def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

varlist = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
dataset[varlist] = dataset[varlist].apply(binary_map)
# Remove uneeded information
dataset.pop('furnishingstatus')
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea
0,13300000,7420,4,2,3,1,0,0,0,1,2,1
1,12250000,8960,4,4,4,1,0,0,0,1,3,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1
4,11410000,7420,4,1,2,1,1,1,0,1,2,0


In [18]:
# Standarize the data to remove size differences
stdScaler = StandardScaler()
num_vars = ['price', 'area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea']
dataset[num_vars] = stdScaler.fit_transform(dataset[num_vars])
# Seperate data into inputs and outputs
x_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']

Y = dataset.pop('price')
X = dataset[x_vars].values
# convert to tensors
X = torch.tensor(X)
Y = torch.tensor(Y)
X, Y

(tensor([[ 1.0467,  1.4034,  1.4218,  1.3782,  1.5177],
         [ 1.7570,  1.4034,  5.4058,  2.5320,  2.6794],
         [ 2.2182,  0.0473,  1.4218,  0.2244,  1.5177],
         ...,
         [-0.7059, -1.3089, -0.5702, -0.9294, -0.8057],
         [-1.0334,  0.0473, -0.5702, -0.9294, -0.8057],
         [-0.5998,  0.0473, -0.5702,  0.2244, -0.8057]], dtype=torch.float64),
 tensor([ 4.5664e+00,  4.0045e+00,  4.0045e+00,  3.9858e+00,  3.5550e+00,
          3.2553e+00,  2.8807e+00,  2.8807e+00,  2.7309e+00,  2.6934e+00,
          2.6934e+00,  2.6297e+00,  2.4312e+00,  2.3938e+00,  2.3938e+00,
          2.3188e+00,  2.3188e+00,  2.2439e+00,  2.2065e+00,  2.1877e+00,
          2.1315e+00,  2.0941e+00,  2.0754e+00,  2.0754e+00,  2.0379e+00,
          2.0192e+00,  1.9780e+00,  1.9443e+00,  1.9443e+00,  1.9443e+00,
          1.9443e+00,  1.9443e+00,  1.8881e+00,  1.8319e+00,  1.7944e+00,
          1.7735e+00,  1.7532e+00,  1.7195e+00,  1.7101e+00,  1.6820e+00,
          1.6633e+00,  1.6446e+00, 

In [19]:
# Split data into training and validation sets
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([319, 500, 286, 454, 413, 261, 341, 314,  11,  55, 183, 361, 480, 320,
         172,  50, 278,  10, 450,  21,  22, 423, 241, 101, 405, 330, 300, 292,
          90, 171, 307, 369, 528, 481, 215, 448, 196,   4,  89, 202, 419, 371,
         398, 453,  77, 233, 194,  12, 273, 251, 532,  28,  74, 472, 342, 332,
         147, 393, 464, 195, 478, 335, 521, 317, 516, 329, 206, 438, 305, 502,
         392, 433, 112, 205, 498, 298, 199, 346, 324, 431, 174, 266,  23, 400,
         131, 463, 111, 106,   3, 218, 114, 167,   0, 437, 509,  75,  17,  73,
         115,  91, 505,  96, 434, 334,  25, 338, 134, 395, 311, 249,  38, 250,
          46,  40,  27,  33,   1, 124, 539,  24, 325, 497, 439,   5, 449, 350,
         181,  88, 301, 349, 274, 403, 492,  42, 229, 372, 275, 510,  44, 306,
         117, 459, 466, 160, 276, 380, 170, 257, 269, 462, 166, 219, 180, 163,
         367, 271, 289,  68,  47, 507, 407, 473, 482, 156, 310, 136,   8, 446,
         270, 373, 447, 536, 390, 523, 159,  19,  15

In [20]:
# Training set
train_X = X[train_indices]
train_Y = Y[train_indices]
# Validation set
val_X = X[val_indices]
val_Y = Y[val_indices]

In [21]:
# Linear model
weights = ['w1', 'w2', 'w3', 'w4', 'w5']
def model(x, w0, w1, w2, w3, w4, w5) :
    return w0 + x[:,0]*w1 + x[:,1]*w2 + x[:,2]*w3 + x[:,3]*w4 + x[:,4]*w5

# Loss function
def loss_fn(y_p, y) :
    squared_diffs = (y_p - y)**2
    return squared_diffs.mean()

In [22]:
# Training Function
def training_fn(epochs, opt, params, x_train, y_train, x_val, y_val) :
    for epoch in range(1, epochs + 1):
        train_p = model(x_train, *params)
        train_loss = loss_fn(train_p, y_train)

        val_p = model(x_val, *params)
        val_loss = loss_fn(val_p, y_val)

        opt.zero_grad()
        train_loss.backward()
        opt.step()

        if epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

    return params

In [23]:
# set up parameters and optimizers
parameters = torch.tensor([1.0, 0.0, 0.0, 0.0, 0.0, 0.0], requires_grad=True)

lr = 0.001
optim_SGD = optim.SGD([parameters], lr=lr)
optim_ADAM = optim.Adam([parameters], lr=lr)

In [24]:
# Training: SGD
training_lr = [0.1, 0.01, 0.001, 0.0001]
for n in training_lr :
    parameters = torch.tensor([1.0, 0.0, 0.0, 0.0, 0.0, 0.0], requires_grad=True)
    optim_SGD = optim.SGD([parameters], lr=n)
    print("Learning Rate:", n)
    training_fn(
        epochs = 5000,
        opt = optim_SGD,
        params = parameters,
        x_train = train_X,
        y_train = train_Y,
        x_val = val_X,
        y_val = val_Y
    )
    print(" ")

Learning Rate: 0.1
Epoch 500, Training loss 0.4716, Validation loss 0.3107
Epoch 1000, Training loss 0.4716, Validation loss 0.3107
Epoch 1500, Training loss 0.4716, Validation loss 0.3107
Epoch 2000, Training loss 0.4716, Validation loss 0.3107
Epoch 2500, Training loss 0.4716, Validation loss 0.3107
Epoch 3000, Training loss 0.4716, Validation loss 0.3107
Epoch 3500, Training loss 0.4716, Validation loss 0.3107
Epoch 4000, Training loss 0.4716, Validation loss 0.3107
Epoch 4500, Training loss 0.4716, Validation loss 0.3107
Epoch 5000, Training loss 0.4716, Validation loss 0.3107
 
Learning Rate: 0.01
Epoch 500, Training loss 0.4716, Validation loss 0.3107
Epoch 1000, Training loss 0.4716, Validation loss 0.3107
Epoch 1500, Training loss 0.4716, Validation loss 0.3107
Epoch 2000, Training loss 0.4716, Validation loss 0.3107
Epoch 2500, Training loss 0.4716, Validation loss 0.3107
Epoch 3000, Training loss 0.4716, Validation loss 0.3107
Epoch 3500, Training loss 0.4716, Validation loss

In [25]:
# Training: ADAM
# Reset parameters
for n in training_lr:
    parameters = torch.tensor([1.0, 0.0, 0.0, 0.0, 0.0, 0.0], requires_grad=True)
    optim_ADAM = optim.Adam([parameters], lr=n)
    print("Learning Rate:", n)
    training_fn(
        epochs = 5000,
        opt = optim_ADAM,
        params = parameters,
        x_train = train_X,
        y_train = train_Y,
        x_val = val_X,
        y_val = val_Y
    )
    print(" ")

Learning Rate: 0.1
Epoch 500, Training loss 0.4716, Validation loss 0.3107
Epoch 1000, Training loss 0.4716, Validation loss 0.3107
Epoch 1500, Training loss 0.4716, Validation loss 0.3107
Epoch 2000, Training loss 0.4716, Validation loss 0.3107
Epoch 2500, Training loss 0.4716, Validation loss 0.3107
Epoch 3000, Training loss 0.4716, Validation loss 0.3107
Epoch 3500, Training loss 0.4716, Validation loss 0.3107
Epoch 4000, Training loss 0.4716, Validation loss 0.3107
Epoch 4500, Training loss 0.4716, Validation loss 0.3107
Epoch 5000, Training loss 0.4716, Validation loss 0.3107
 
Learning Rate: 0.01
Epoch 500, Training loss 0.4716, Validation loss 0.3107
Epoch 1000, Training loss 0.4716, Validation loss 0.3107
Epoch 1500, Training loss 0.4716, Validation loss 0.3107
Epoch 2000, Training loss 0.4716, Validation loss 0.3107
Epoch 2500, Training loss 0.4716, Validation loss 0.3107
Epoch 3000, Training loss 0.4716, Validation loss 0.3107
Epoch 3500, Training loss 0.4716, Validation loss